# Julia for FEM: ``Ferrite.jl``
Ferrite is a finite element toolbox that provides functionalities to implement finite element analysis in Julia. The aim is to be general and to keep mathematical abstractions. The main functionalities of the package include:
- Facilitate integration using different quadrature rules.
- Define different finite element interpolations.
- Evaluate shape functions, derivatives of shape functions etc. for the different interpolations and quadrature rules.
- Evaluate functions and derivatives in the finite element space.
- Generate simple grids.
- Export grids and solutions to VTK.

This package used to be called ``JuAFEM.jl``.

## Installation
You can install Ferrite from the Pkg REPL (press ``]`` in the Julia REPL to enter ``pkg>`` mode):
```
pkg> add Ferrite
```

## Documentation & Tutorials
The documentation and tutorials for the ``Ferrite.jl`` package can all be found on the [github page](https://ferrite-fem.github.io/Ferrite.jl/stable/) of the package. The tutorials are mostly focussed on mechanical and fluid dynamic problems, but can be adapted for our power engineering purposes.

## Integration with ``Gmsh``
We can directly use the geometries defined using Gmsh in ``Ferrite.jl`` using the [FerriteGmsh.jl](https://github.com/Ferrite-FEM/FerriteGmsh.jl) plugin. The model can be loaded directly with
```julia
using Ferrite, FerriteGmsh

grid = saved_file_to_grid("path/to/model.msh")
```
Named physical groups (e.g. domains or boundaries) can be accessed in ``Ferrite`` to assign properties, such as material properties and boundary conditions. An example of this is given below.

## Visualization
Visualization of the FEM results can be done in two ways. 
- Using an external program for viewing ``.vtk`` files, such as [ParaView](https://www.paraview.org/). This seems to be the preferred method.
- Using the [FerriteViz.jl](https://github.com/Ferrite-FEM/FerriteViz.jl) package for plotting with [Makie.jl](https://github.com/JuliaPlots/Makie.jl).

# Example: High-Voltage Cable

In [1]:
import Gmsh: gmsh

using Ferrite, FerriteGmsh
using SparseArrays

## Define Geometry

In [2]:
R_cond = 19.1e-3;
R_ins  = 18.4e-3;
R_sh   = 1e-3;
R_jac  = 8e-3;

r_cond = R_cond;             # Conductor
r_ins  = r_cond + R_cond;    # Insulator
r_sh   = r_ins + R_sh;       # Sheath
r_jac  = r_sh + R_jac;       # Jacket

V = 245e3 * sqrt(2 / 3); # Operating voltage

# Mesh density
mshd_cond = R_cond / 20; 
mshd_ins  = R_ins / 10;
mshd_sh   = R_sh / 2;
mshd_jac  = R_jac / 2;

In [3]:
gmsh.finalize()
gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)
gmsh.model.add("ferrite_example")

geo = gmsh.model.geo;

## Points
geo.addPoint(0, 0, 0, 1, 1)
geo.addPoint(+r_cond, 0, 0, mshd_cond, 2)
geo.addPoint(-r_cond, 0, 0, mshd_cond, 3)
geo.addPoint(+r_ins, 0, 0, mshd_sh, 4)
geo.addPoint(-r_ins, 0, 0, mshd_sh, 5)
geo.addPoint(+r_sh, 0, 0, mshd_sh, 6)
geo.addPoint(-r_sh, 0, 0, mshd_sh, 7)
geo.addPoint(+r_jac, 0, 0, mshd_jac, 8)
geo.addPoint(-r_jac, 0, 0, mshd_jac, 9)

## Curves
geo.addCircleArc(2, 1, 3, 1)
geo.addCircleArc(3, 1, 2, 2)
geo.addCircleArc(4, 1, 5, 3)
geo.addCircleArc(5, 1, 4, 4)
geo.addCircleArc(6, 1, 7, 5)
geo.addCircleArc(7, 1, 6, 6)
geo.addCircleArc(8, 1, 9, 7)
geo.addCircleArc(9, 1, 8, 8)

## Surfaces
geo.addCurveLoop([1, 2], 1)
geo.addCurveLoop([3, 4], 2)
geo.addCurveLoop([5, 6], 3)
geo.addCurveLoop([7, 8], 4)

geo.addPlaneSurface([1], 1)
geo.addPlaneSurface([2, 1], 2)
geo.addPlaneSurface([3, 1, 2], 3)
geo.addPlaneSurface([4, 1, 2, 3], 4)

## Define domains
gmsh.model.geo.synchronize()

geo.addPhysicalGroup(2, [1], 1) # Conductor
geo.addPhysicalGroup(2, [2], 2) # Dielectric
geo.addPhysicalGroup(2, [3], 3) # Sheath
geo.addPhysicalGroup(2, [4], 4) # Jacket
gmsh.model.setPhysicalName(2, 1, "Dielectric")
gmsh.model.setPhysicalName(2, 2, "Conductor")
gmsh.model.setPhysicalName(2, 3, "Sheath")
gmsh.model.setPhysicalName(2, 4, "Jacket")

geo.addPhysicalGroup(1, [1, 2], 1) # Conductor boundary
geo.addPhysicalGroup(1, [3, 4], 2) # Sheath boundary
geo.addPhysicalGroup(0, [2, 3], 1) # Conductor nodes
geo.addPhysicalGroup(0, [4, 5], 2) # Sheath nodes

gmsh.model.setPhysicalName(1, 1, "D1")
gmsh.model.setPhysicalName(1, 2, "D2")
gmsh.model.setPhysicalName(0, 1, "D1p")
gmsh.model.setPhysicalName(0, 2, "D2p")

# Generate mesh and save
gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)

gmsh.write("geo/ferrite_example.msh")

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 20%] Meshing curve 2 (Circle)
Info    : [ 30%] Meshing curve 3 (Circle)
Info    : [ 40%] Meshing curve 4 (Circle)
Info    : [ 50%] Meshing curve 5 (Circle)
Info    : [ 70%] Meshing curve 6 (Circle)
Info    : [ 80%] Meshing curve 7 (Circle)
Info    : [ 90%] Meshing curve 8 (Circle)
Info    : Done meshing 1D (Wall 0.000980625s, CPU 0.000969s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 30%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : [ 80%] Meshing surface 4 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.300151s, CPU 0.299193s)
Info    : 13885 nodes 28877 elements
Info    : Writing 'geo/ferrite_example.msh'...
Info    : Done writing 'geo/ferrite_example.msh'


Error   : Gmsh has not been initialized


## Ferrite

In [4]:
# Load the mesh using FerriteGmsh
grid = saved_file_to_grid("geo/ferrite_example.msh");

Info    : Reading 'geo/ferrite_example.msh'...
Info    : 21 entities
Info    : 13884 nodes
Info    : 28302 elements
Info    : Done reading 'geo/ferrite_example.msh'


In [5]:
# Create finite element interpolation and quadrature rule
# The FE interpolation & quadrature have a variable order (set to 2 for quadratic elements)
# Because the geometry is built from linear triangular elements, the geometry interpolation must have order 1
dim   = 2
order = 2;
ip_fe  = Lagrange{dim, RefTetrahedron, order}()
ip_geo = Lagrange{dim, RefTetrahedron, 1}()
qr = QuadratureRule{dim, RefTetrahedron}(order)
cellvalues = CellScalarValues(qr, ip_fe, ip_geo);

# Create handler for the degrees of freedom, and define the scalar potential field
dh   = DofHandler(grid);
push!(dh, :V, 1, ip_fe)
close!(dh)

DofHandler
  Fields:
    :V, interpolation: Lagrange{2, RefTetrahedron, 2}(), dim: 1
  Dofs per cell: 6
  Total dofs: 55457

In [6]:
# Define the boundary conditions using a constraint handler
ch = ConstraintHandler(dh);

# Non-homogeneous Dirichlet condition on boundary D1 (conductor), with value V
dbc_V = Dirichlet(
    :V,
    getfaceset(grid, "D1"),
    (x,t) -> V;
);
# Homogeneous Dirichlet condition on boundary D2 (sheath)
dbc_0 = Dirichlet(
    :V,
    getfaceset(grid, "D2"),
    (x,t) -> 0;
)

# Add boundary conditions to the constraint handler and update it
add!(ch, dbc_V)
add!(ch, dbc_0)

close!(ch)
update!(ch, 0.0); # Since the BCs do not depend on time, update them once at t = 0.0

In [7]:
# Element assembly: computes the elementary matrix contributions Ke and fe
function assemble_element!(Ke::Matrix, fe::Vector, cellvalues::CellScalarValues)
    n_basefuncs = getnbasefunctions(cellvalues)
    # Reset to 0
    fill!(Ke, 0)
    fill!(fe, 0)
    # Loop over quadrature points
    for q_point in 1:getnquadpoints(cellvalues)
        # Get the quadrature weight
        dΩ = getdetJdV(cellvalues, q_point)
        # Loop over test shape functions
        for i in 1:n_basefuncs
            v  = shape_value(cellvalues, q_point, i)
            ∇v = shape_gradient(cellvalues, q_point, i)
            
            # Add contribution to fe
            # Not necessary for this problem, since there is no space charge
            #fe[i] += 0 * δu * dΩ
            
            # Loop over trial shape functions
            for j in 1:n_basefuncs
                ∇u = shape_gradient(cellvalues, q_point, j)
                # Add contribution to Ke
                Ke[i, j] += (∇v ⋅ ∇u) * dΩ
            end
        end
    end
    return Ke, fe
end

# Global assembly: computes global matrix K and f using element contributions
#  This is quite fast because of the use of a pre-allocated sparse matrix pattern.
function assemble_global(cellvalues::CellScalarValues, K::SparseMatrixCSC, dh::DofHandler)
    # Allocate the element stiffness matrix and element force vector
    n_basefuncs = getnbasefunctions(cellvalues)
    Ke = zeros(n_basefuncs, n_basefuncs)
    fe = zeros(n_basefuncs)
    # Allocate global force vector f
    f = zeros(ndofs(dh))
    
    # Create an assembler
    assembler = start_assemble(K, f)
    # Loop over all cels
    for cell in CellIterator(dh)
        # Reinitialize cellvalues for this cell
        reinit!(cellvalues, cell)
        # Compute element contribution
        assemble_element!(Ke, fe, cellvalues)
        
        # Assemble Ke and fe into K and f
        assemble!(assembler, celldofs(cell), Ke, fe)
    end
    return K, f
end

assemble_global (generic function with 1 method)

In [8]:
# Create sparsity pattern from mesh data and assemble the linear system
K = create_sparsity_pattern(dh);
K, f = assemble_global(cellvalues, K, dh);

# Apply the boundary conditions
apply!(K, f, ch)

# Solve the linear system
u = K \ f;

## Post-processing

In [9]:
# Compute the electric field on the elements
#    E = -grad V
function compute_E(cellvalues::CellScalarValues{dim,T}, dh::DofHandler, a) where {dim,T}
    n = getnbasefunctions(cellvalues)
    cell_dofs = zeros(Int, n)
    nqp = getnquadpoints(cellvalues)

    # Allocate storage for the fluxes to store
    E = [Ferrite.Vec{2,T}[] for _ in 1:getncells(dh.grid)]

    for (cell_num, cell) in enumerate(CellIterator(dh))
        E_cell = E[cell_num]
        celldofs!(cell_dofs, dh, cell_num)
        ae = a[cell_dofs]
        reinit!(cellvalues, cell)

        for E_point in 1:nqp
            E_qp = - function_gradient(cellvalues, E_point, ae)
            push!(E_cell, E_qp)
        end
    end
    return E
end

# The function above returns values of E for each quadrature point, averaging these gives the value on each element
E_gp = compute_E(cellvalues, dh, u);
E = collect(mean(norm.(E_gp[i])) for i = 1:getncells(dh.grid));

In [10]:
# Export the electric potential V (u) and and electric field E to a VTK file for visualization using ParaView
vtk_grid("images/ferrite_example", dh) do vtk
    vtk_point_data(vtk, dh, u)
    vtk_cell_data(vtk, E, "E")
end

1-element Vector{String}:
 "images/ferrite_example.vtu"

## Electric Potential
![Results: Electric Potential](images/ferrite_ex_V.png)

## Electric Field Strength
![Results: Electric Field Strength](images/ferrite_ex_E.png)

# Interactive Visualization using WGLMakie

In [11]:
import WGLMakie
import JSServe #for interactive display of WGLMakie
import FerriteViz
JSServe.Page()

In [12]:
dh_grad, u_grad = FerriteViz.interpolate_gradient_field(dh, u, :V)
plotter_grad = FerriteViz.MakiePlotter(dh_grad,u_grad)
FerriteViz.solutionplot(plotter_grad)